<!-- WARNING: THIS FILE WAS AUTOGENERATED! DO NOT EDIT! -->

In [ ]:
import requests 
from bs4 import BeautifulSoup
from nbdev.showdoc import *
from deliveroo_editions.selenium_utils import *
from deliveroo_editions.deliveroo_utils import *
import pandas as pd
import datetime

We can find all of Deliveroo's rated kitchens at the following url:

In [ ]:
url = "https://ratings.food.gov.uk/enhanced-search/en-GB/%22deliveroo%22/%5E/Relevance/0/%5E/%5E/0/1/50"

Lets parse the webpage and get the addresses for each of the restaurants listed:

In [ ]:
response = requests.get(url)

if response.status_code == 200:
    html_content = response.content
    soup = BeautifulSoup(html_content, 'html.parser')

In [ ]:
businesses = soup.find_all('div', class_='ResultsBusinessDetails')

In [ ]:
type(businesses[0])

bs4.element.Tag

In [ ]:
def extract_address(business:"bs4.element.Tag"  # div of class='ResultsBusinessDetails'
                   ):
                       "Extracts address string from div"
                       return business.find('div', class_='ResultsBusinessAddress').text.strip().replace('\r\n', ' ')

def extract_postcode(business:"bs4.element.Tag"  # div of class='ResultsBusinessDetails'
                    ):
                        "Extracts address string from div"
                        return business.find('div', class_='ResultsBusinessPostcode').text.strip().replace('\r\n', ' ')

In [ ]:
addresses = [{"address": extract_address(business), "postcode": extract_postcode(business)} for business in businesses]
assert len(addresses) == 31

In [ ]:
addresses

[{'address': 'Unit 2 Editions Battersea 15a Parkfield Industrial Estate Culvert Place London',
  'postcode': 'SW115BA'},
 {'address': 'Deliveroo Unit 4 Roman Way Industrial Estate 149 Roman Way Islington',
  'postcode': 'N7 8XH'},
 {'address': 'Unit 3-4 Restwell House Coldhams Road  Cambridge Cambridgeshire',
  'postcode': 'CB1 3EW'},
 {'address': 'Unit 1 Saxon Works 22 Olive Road Hove', 'postcode': 'BN3 7GY'},
 {'address': 'Unit 1 Scott Hall Mills Scott Hall Street Meanwood Leeds',
  'postcode': 'LS7 2HT'},
 {'address': '20 Acton Park Estate The Vale Acton Ealing',
  'postcode': 'W3 7QE'},
 {'address': '33 Acton Park Estate The Vale Acton London',
  'postcode': 'W3 7QE'},
 {'address': 'Unit 1 Scott Hall Mills Scott Hall Street Meanwood',
  'postcode': 'LS7 2HT'},
 {'address': '86 North Western Street Ardwick Manchester',
  'postcode': 'M12 6DY'},
 {'address': 'Railway Arches 89 To 90  Enid Street London',
  'postcode': 'SE16 3RA'},
 {'address': 'Ground Floor and Basement and Rear Yard

Now we have the ~30 addresses, we can search Deliveroo's site for all the editions kitchens based at these addresses:

In [ ]:
address_strings = [address['address'] + " " + address['postcode'] for address in addresses]
address_strings

['Unit 2 Editions Battersea 15a Parkfield Industrial Estate Culvert Place London SW115BA',
 'Deliveroo Unit 4 Roman Way Industrial Estate 149 Roman Way Islington N7 8XH',
 'Unit 3-4 Restwell House Coldhams Road  Cambridge Cambridgeshire CB1 3EW',
 'Unit 1 Saxon Works 22 Olive Road Hove BN3 7GY',
 'Unit 1 Scott Hall Mills Scott Hall Street Meanwood Leeds LS7 2HT',
 '20 Acton Park Estate The Vale Acton Ealing W3 7QE',
 '33 Acton Park Estate The Vale Acton London W3 7QE',
 'Unit 1 Scott Hall Mills Scott Hall Street Meanwood LS7 2HT',
 '86 North Western Street Ardwick Manchester M12 6DY',
 'Railway Arches 89 To 90  Enid Street London SE16 3RA',
 'Ground Floor and Basement and Rear Yard 70 New Oxford Street  London WC1A 1EU',
 'Units 5-7 Yorkton Street Hoxton  Hackney London E2 8NH',
 'Unit 12 St Josephs Business Park St Josephs Close Hove BN3 7HG',
 'Units 29 To 30 Clifton Road  Cambridge Cambridgeshire CB1 7EB',
 'Arch 114 Randall Road London LONDON SE11 5JR',
 'Unit 2 271 Merton Road Lon

In [ ]:
attempts = 0
while attempts < 2:
    try:
        editions_locations = get_editions_locations_near_addresses(address_strings, driver)
        attempts = 2
    except:
        attempts += 1


KeyboardInterrupt



In [ ]:
editions_locations

We can now get all the editions kitchens based at all of these sites:

In [ ]:
editions = get_restaurants_from_editions_location(editions_locations)

In [ ]:
editions_df = pd.DataFrame(editions)

In [ ]:
columns_subset = ['name', 'location', 'edition']

In [ ]:
editions_df = editions_df.drop_duplicates(columns_subset).reset_index(drop=True)

In [ ]:
current_time = datetime.datetime.now()

In [ ]:
filename = 'data/editions_' + current_time.strftime("%Y_%m_%d_%Hhr") + ".csv"

In [ ]:
editions_df.to_csv(filename, encoding="utf-8")